In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from pylab import rcParams
from scipy.ndimage.interpolation import rotate, zoom
import scipy as sp
import cv2
%matplotlib inline
import sys
from tqdm import tqdm
import argparse
from IPython.display import clear_output

sys.path.append("/home/msmith/misc/pcaRotate/") # Helper function
from pcaRotate import main as rotate
from pcaRotate import getRed
from pylab import rcParams
rcParams["figure.figsize"] = 30,25


def show():
    plt.title(wp)
    plt.subplot(161)
    plt.imshow(orig)
    plt.subplot(162)
    plt.imshow(mask)
    plt.subplot(163)
    plt.imshow(maskRot,cmap=cm.gray)
    plt.subplot(164)
    plt.imshow(red,cmap=cm.gray)
    #plt.subplot(165)
    #plt.imshow(origRot)
    plt.subplot(165)
    plt.imshow(croppedHead)
    plt.show()

In [ ]:
if __name__ == "__main__": 
    imgPaths = glob.glob("/home/msmith/kaggle/whale/imgs/whale*/m1_ss_*")
    testPaths = glob.glob("/home/msmith/kaggle/whale/imgs/test*/m1_ss_*")
    print("Make sure image is in RGB order not BGR")
    imgPaths.sort()
    imgPaths += testPaths
    maskPath = "m1_ss_"
    dstPath = "head_ss_"
    cantSave = []
    aspectRatio = 1.4
    h = 400
    w = int(h*aspectRatio)
    count = 0
    for path in tqdm(imgPaths):
        
        orig, mask = [cv2.imread(x)[:,:,::-1] for x in [path.replace(maskPath,"w1_"),path]]
        croppedHead, maskRot, red = rotate(orig=orig,mask=mask,ellipseThresh=10,redThresh=[0.08,0.13],cntThresh=0.01,pad=20,aspectRatio=aspectRatio)
        
        wp = path.replace(maskPath,dstPath)
        
        if croppedHead.size == 0:
            print("couldn't save {0}".format(wp))
            cantSave.append(wp)
            show()
        else:
            croppedHead = cv2.resize(croppedHead,(w,h),interpolation=cv2.INTER_LINEAR)
            cv2.imwrite(wp,croppedHead)
        
        if np.random.uniform() < 0.01:
            show()
            count += 1
            
        if count == 10:
            clear_output()
            count = 0


  0%|          | 0/11468 [00:00<?, ?it/s]

Make sure image is in RGB order not BGR


  0%|          | 24/11468 [00:23<2:33:03,  1.25it/s]